In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from time import time
import pandas as pd
import re
from scipy import sparse

In [39]:
def get_distribution_matrix(corpus, ngram, mindf):
    vectorizer = CountVectorizer(stop_words='english', ngram_range=(ngram,ngram),min_df=mindf)
    f = vectorizer.fit_transform(corpus)
#     print(f.shape)
    l = len(vectorizer.get_feature_names())
    return np.divide(f, l)

In [24]:
def calc_kl(p, q):
    return sum([p[i]*(np.log2(p[i]/q[i])) for i in range(len(p))])

In [25]:
def create_df_time(df, time):
    return df[df.PublishDate.str[:7] == time]

In [3]:
df = pd.read_csv('../../data/shakespare_william_works_preprocessed.tsv', sep = '\t')

In [7]:
doc = []
for i in df.Text.tolist():
    #Remove some non-ascii characters and 'aa's
    i = re.sub(r'aA|aa', 'a', i)
    i = re.sub(r'\\xe2........|\\xc|\\xa|\\n|[0123456789*_]', '', i)
    doc.append(i)  

In [23]:
get_distribution_matrix(doc, 1)

(3586, 24164)


<3586x24164 sparse matrix of type '<class 'numpy.float64'>'
	with 1596876 stored elements in Compressed Sparse Row format>

In [26]:
timelist = df.PublishDate.drop_duplicates().tolist()
timelist = [str(i)[:7] for i in timelist]
timelist = sorted(list(set(timelist)))

In [40]:
cnt = 0
kl_all = []
t0 = time()
min_df = 3
for t in timelist:
    df_t = create_df_time(df, t)
#     print(len(df_t))
    if len(df_t) > min_df:
        doc = []
        for i in df_t.Text.tolist():
            #Remove some non-ascii characters and 'aa's
            i = re.sub(r'aA|aa', 'a', i)
            i = re.sub(r'\\xe2........|\\xc|\\xa|\\n|[0123456789*_]', '', i)
            doc.append(i)  
        m = get_distribution_matrix(doc,1,min_df)
        std = sparse.csr_matrix.mean(m, axis=0)
        std = np.asarray(std)[0]
        kl_month = []
        for row in m.toarray():
            kl = calc_kl(row, std)
            if not np.isnan(kl):
                kl_month.append(kl)
        kl_all.append(np.average(kl_month))
print("done in %0.3fs." % (time() - t0))

/Users/jingy/anaconda/envs/python35/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


done in 37.685s.


In [41]:
print (len(timelist), len(kl_all))

102 78
